# Kruskal Tensors

```
Copyright 2022 National Technology & Engineering Solutions of Sandia,
LLC (NTESS). Under the terms of Contract DE-NA0003525 with NTESS, the
U.S. Government retains certain rights in this software.
```

Kruskal format is a decomposition of a tensor $\mathcal{X}$ as the sum of the outer products as the columns of matrices. For example, we might write:

$
\mathcal{X} = \sum_{r} a_r \circ b_r \circ c_r
$

where a subscript denotes column index and a circle denotes outer product. In other words, the tensor X is built from the columns of the matrices A,B, and C. It's often helpful to explicitly specify a weight for each outer product, which we do here:

$
\mathcal{X} = \sum_{r} \lambda_r \, a_r \circ b_r \circ c_r
$

The **`ktensor`** class stores the components of the tensor $\mathcal{X}$ and can perform many operations, e.g., **`ttm`**, without explicitly forming the tensor $\mathcal{X}$.

## Contents

* [Kruskal tensor format via ktensor](#Kruskal-tensor-format-via-ktensor)
* [Specifying weights in a ktensor](#Specifying-weights-in-a-ktensor)
* [Creating a one-dimensional ktensor](#Creating-a-one-dimensional-ktensor)
* [Constituent parts of a ktensor](#Constituent-parts-of-a-ktensor)
* [Creating a ktensor from its constituent parts](#Creating-a-ktensor-from-its-constituent-parts)
* [Creating an empty ktensor](#Creating-an-empty-ktensor)
* [Use full or tensor to convert a ktensor to a tensor
](#Use-full-or-tensor-to-convert-a-ktensor-to-a-tensor)
* [Use double to convert a ktensor to a multidimensional array](#Use-double-to-convert-a-ktensor-to-a-multidimensional-array)
* [Use tendiag or sptendiag to convert a ktensor to a ttensor](#Use-tendiag-or-sptendiag-to-convert-a-ktensor-to-a-ttensor)
* [Use ndims and size for the dimensions of a ktensor](#Use-ndims-and-size-for-the-dimensions-of-a-ktensor)
* [Subscripted reference for a ktensor](#Subscripted-reference-for-a-ktensor)
* [Subscripted assignment for a ktensor](#Subscripted-assignment-for-a-ktensor)
* [Use end for the last array index](#Use-end-for-the-last-array-index)
* [Adding and subtracting ktensors](#Adding-and-subtracting-ktensors)
* [Basic operations with a ktensor](#Basic-operations-with-a-ktensor)
* [Use permute to reorder the modes of a ktensor](#Use-permute-to-reorder-the-modes-of-a-ktensor)
* [Use arrange to normalize the factors of a ktensor](#Use-arrange-to-normalize-the-factors-of-a-ktensor)
* [Use fixsigns for sign indeterminacies in a ktensor](#Use-fixsigns-for-sign-indeterminacies-in-a-ktensor)
* [Use ktensor to store the 'skinny' SVD of a matrix](#Use-ktensor-to-store-the-'skinny'-SVD-of-a-matrix)
* [Displaying data](#Displaying-data)

## Kruskal tensor format via ktensor

In [1]:
import pyttb as ttb
import numpy as np

Kruskal format stores a tensor as a sum of rank-1 outer products. For example, consider a tensor of the following form:

$
X = a_1 \circ b_1 \circ c_1 + a_2 \circ b_2 \circ c_2
$

This can be stored in Kruskal form as follows.

In [2]:
np.random.seed(0)
A = np.random.rand(4,2) # First column is a_1, second is a_2.
B = np.random.rand(3,2) # Likewise for B.
C = np.random.rand(2,2) # Likewise for C.
X = ttb.ktensor.from_factor_matrices([A,B,C]) # Create the ktensor.
X

ktensor of shape 4 x 3 x 2
weights=[1. 1.]
factor_matrices[0] =
[[0.5488135  0.71518937]
 [0.60276338 0.54488318]
 [0.4236548  0.64589411]
 [0.43758721 0.891773  ]]
factor_matrices[1] =
[[0.96366276 0.38344152]
 [0.79172504 0.52889492]
 [0.56804456 0.92559664]]
factor_matrices[2] =
[[0.07103606 0.0871293 ]
 [0.0202184  0.83261985]]

For Kruskal format, there can be any number of matrices, but every matrix must have the same number of columns. The number of rows can vary.

In [3]:
Y = ttb.ktensor.from_factor_matrices(
    [np.random.rand(4,1), np.random.rand(2,1), np.random.rand(3,1)]
) # Another ktensor.
Y

ktensor of shape 4 x 2 x 3
weights=[1.]
factor_matrices[0] =
[[0.77815675]
 [0.87001215]
 [0.97861834]
 [0.79915856]]
factor_matrices[1] =
[[0.46147936]
 [0.78052918]]
factor_matrices[2] =
[[0.11827443]
 [0.63992102]
 [0.14335329]]

## Specifying weights in a ktensor

Weights for each rank-1 tensor can be specified by passing in a column vector. For example: 

$
X = \lambda_1 \, a_1 \circ b_1 \circ c_1 + \lambda_2 \, a_2 \circ b_2 \circ c_2 $

## **TODO** : Propose only using the instantiation of X once here, since X is not modified going forward until section ***[Subscripted assignment for a ktensor](#Subscripted-assignment-for-a-ktensor)***, and it would save space in many of the subsequent code blocks without confusing the user.

In [4]:
np.random.seed(0)
A = np.random.rand(4,2) # Create some data.
B = np.random.rand(3,2)
C = np.random.rand(2,2)
weights = np.array([5.0, 0.25])
X = ttb.ktensor.from_data(weights, [A,B,C]) # Create the ktensor.
X

ktensor of shape 4 x 3 x 2
weights=[5.   0.25]
factor_matrices[0] =
[[0.5488135  0.71518937]
 [0.60276338 0.54488318]
 [0.4236548  0.64589411]
 [0.43758721 0.891773  ]]
factor_matrices[1] =
[[0.96366276 0.38344152]
 [0.79172504 0.52889492]
 [0.56804456 0.92559664]]
factor_matrices[2] =
[[0.07103606 0.0871293 ]
 [0.0202184  0.83261985]]

## Creating a one-dimensional ktensor

In [5]:
np.random.seed(0)
Y = ttb.ktensor.from_factor_matrices(np.random.rand(4,5)) # A one-dimensional ktensor.
Y

ktensor of shape 4
weights=[1. 1. 1. 1. 1.]
factor_matrices[0] =
[[0.5488135  0.71518937 0.60276338 0.54488318 0.4236548 ]
 [0.64589411 0.43758721 0.891773   0.96366276 0.38344152]
 [0.79172504 0.52889492 0.56804456 0.92559664 0.07103606]
 [0.0871293  0.0202184  0.83261985 0.77815675 0.87001215]]

## Constituent parts of a ktensor

In [6]:
# np.random.seed(0)
# A = np.random.rand(4,2) # Create some data.
# B = np.random.rand(3,2)
# C = np.random.rand(2,2)
# weights = np.array([5.0, 0.25])
# X = ttb.ktensor.from_data(weights, [A,B,C]) # Create the ktensor.

X.weights # Weights or multipliers.

array([5.  , 0.25])

In [7]:
X.factor_matrices # Cell array of matrices.

[array([[0.5488135 , 0.71518937],
        [0.60276338, 0.54488318],
        [0.4236548 , 0.64589411],
        [0.43758721, 0.891773  ]]),
 array([[0.96366276, 0.38344152],
        [0.79172504, 0.52889492],
        [0.56804456, 0.92559664]]),
 array([[0.07103606, 0.0871293 ],
        [0.0202184 , 0.83261985]])]

## Creating a ktensor from its constituent parts

In [8]:
# np.random.seed(0)
# A = np.random.rand(4,2) # Create some data.
# B = np.random.rand(3,2)
# C = np.random.rand(2,2)
# weights = np.array([5.0, 0.25])
# X = ttb.ktensor.from_data(weights, [A,B,C]) # Create the ktensor.

Y = ttb.ktensor.from_data(X.weights, X.factor_matrices) # Recreate X.
Y

ktensor of shape 4 x 3 x 2
weights=[5.   0.25]
factor_matrices[0] =
[[0.5488135  0.71518937]
 [0.60276338 0.54488318]
 [0.4236548  0.64589411]
 [0.43758721 0.891773  ]]
factor_matrices[1] =
[[0.96366276 0.38344152]
 [0.79172504 0.52889492]
 [0.56804456 0.92559664]]
factor_matrices[2] =
[[0.07103606 0.0871293 ]
 [0.0202184  0.83261985]]

## Creating an empty ktensor

In [9]:
Z = ttb.ktensor() # Empty ktensor.
Z

ktensor of shape 
weights=[]

## Use full or tensor to convert a ktensor to a tensor

In [10]:
# np.random.seed(0)
# A = np.random.rand(4,2) # Create some data.
# B = np.random.rand(3,2)
# C = np.random.rand(2,2)
# weights = np.array([5.0, 0.25])
# X = ttb.ktensor.from_data(weights, [A,B,C]) # Create the ktensor.

X.full() # Converts to a tensor.

tensor of shape 4 x 3 x 2
data[0, :, :] = 
[[0.19381804 0.11054766]
 [0.16256856 0.12266212]
 [0.12514704 0.16930925]]
data[1, :, :] = 
[[0.21086124 0.10221034]
 [0.17577751 0.10823074]
 [0.13259822 0.13959484]]
data[2, :, :] = 
[[0.1504007  0.09282405]
 [0.12657497 0.10501592]
 [0.09849813 0.1487711 ]]
data[3, :, :] = 
[[0.15722304 0.11380622]
 [0.13332549 0.13320036]
 [0.10626643 0.19694404]]

In [11]:
ttb.tensor.from_tensor_type(X) # Same as above.

tensor of shape 4 x 3 x 2
data[0, :, :] = 
[[0.19381804 0.11054766]
 [0.16256856 0.12266212]
 [0.12514704 0.16930925]]
data[1, :, :] = 
[[0.21086124 0.10221034]
 [0.17577751 0.10823074]
 [0.13259822 0.13959484]]
data[2, :, :] = 
[[0.1504007  0.09282405]
 [0.12657497 0.10501592]
 [0.09849813 0.1487711 ]]
data[3, :, :] = 
[[0.15722304 0.11380622]
 [0.13332549 0.13320036]
 [0.10626643 0.19694404]]

## Use double to convert a ktensor to a multidimensional array

In [12]:
# np.random.seed(0)
# A = np.random.rand(4,2) # Create some data.
# B = np.random.rand(3,2)
# C = np.random.rand(2,2)
# weights = np.array([5.0, 0.25])
# X = ttb.ktensor.from_data(weights, [A,B,C]) # Create the ktensor.

X.double() # Converts to an array.

array([[[0.19381804, 0.11054766],
        [0.16256856, 0.12266212],
        [0.12514704, 0.16930925]],

       [[0.21086124, 0.10221034],
        [0.17577751, 0.10823074],
        [0.13259822, 0.13959484]],

       [[0.1504007 , 0.09282405],
        [0.12657497, 0.10501592],
        [0.09849813, 0.1487711 ]],

       [[0.15722304, 0.11380622],
        [0.13332549, 0.13320036],
        [0.10626643, 0.19694404]]])

## Use tendiag or sptendiag to convert a ktensor to a ttensor

A ktensor can be regarded as a ttensor with a diagonal core.

## TODO: tendiag() is bugging for me. [See issue](https://github.com/sandialabs/pyttb/issues/148)

In [13]:
# np.random.seed(0)
# A = np.random.rand(4,2) # Create some data.
# B = np.random.rand(3,2)
# C = np.random.rand(2,2)
# weights = np.array([5.0, 0.25])
# X = ttb.ktensor.from_data(weights, [A,B,C]) # Create the ktensor.

R = len(X.weights) # Number of factors in X.
# core = ttb.tendiag(X.weights, ((R, ) * X.ndims)) # Create a diagonal core.
# Y = ttb.ttensor.from_data(core, X.factor_matrices)

"""
--- Simulating correct tendiag() functionality in lieu of line 9 ---
"""
core = ttb.tensor.from_data(
    np.array([
        [
            [5.0, 0.0],
            [0.0, 0.0]
        ],
        [
            [0.0, 0.0],
            [0.0, 0.25]
        ]
    ])
)
Y = ttb.ttensor.from_data(core, X.factor_matrices)
Y

Tensor of shape: (4, 3, 2)
	Core is a 	tensor of shape 2 x 2 x 2
	data[0, :, :] = 
	[[5. 0.]
	 [0. 0.]]
	data[1, :, :] = 
	[[0.   0.  ]
	 [0.   0.25]]
	U[0] = 
		[[0.5488135  0.71518937]
		 [0.60276338 0.54488318]
		 [0.4236548  0.64589411]
		 [0.43758721 0.891773  ]]
	U[1] = 
		[[0.96366276 0.38344152]
		 [0.79172504 0.52889492]
		 [0.56804456 0.92559664]]
	U[2] = 
		[[0.07103606 0.0871293 ]
		 [0.0202184  0.83261985]]

In [14]:
(X.full() - Y.full()).norm() # They are the same.

8.554842949272357e-17

## TODO: confirm that it's fine to convert `core` into a tensor-type; bug-> core of type sptensor is disallowed due to `ValueError: TTENSOR doesn't yet support generic cores, only tensor` when trying to instance Y below

In [15]:
core = ttb.sptendiag(X.weights, ((R, ) * X.ndims)) # Sparse diagonal core.
"""
--- Converting core to tensor else ValueError for Y-instancing ---
"""
core = ttb.tensor.from_tensor_type(core) # <--- CONVERSION HERE
Y = ttb.ttensor.from_data(core, X.factor_matrices) # Assemble the ttensor
Y

Tensor of shape: (4, 3, 2)
	Core is a 	tensor of shape 2 x 2 x 2
	data[0, :, :] = 
	[[5. 0.]
	 [0. 0.]]
	data[1, :, :] = 
	[[0.   0.  ]
	 [0.   0.25]]
	U[0] = 
		[[0.5488135  0.71518937]
		 [0.60276338 0.54488318]
		 [0.4236548  0.64589411]
		 [0.43758721 0.891773  ]]
	U[1] = 
		[[0.96366276 0.38344152]
		 [0.79172504 0.52889492]
		 [0.56804456 0.92559664]]
	U[2] = 
		[[0.07103606 0.0871293 ]
		 [0.0202184  0.83261985]]

In [16]:
(X.full() - Y.full()).norm() # They are the same.

8.554842949272357e-17

## Use ndims and size for the dimensions of a ktensor

In [17]:
X.ndims # Number of dimensions.

3

In [18]:
X.shape # Tuple of the sizes.

(4, 3, 2)

In [19]:
X.shape[1] # Size of the 2nd mode.

3

## Subscripted reference for a ktensor

In [20]:
X[0][0][0] # Assemble the (0,0,0) element

0.5488135039273248

In [21]:
X.weights[1] # Weight of the 2nd factor.

0.25

In [22]:
X.factor_matrices[1] # Extract a matrix.

array([[0.96366276, 0.38344152],
       [0.79172504, 0.52889492],
       [0.56804456, 0.92559664]])

In [23]:
X[1] # Same as above.

array([[0.96366276, 0.38344152],
       [0.79172504, 0.52889492],
       [0.56804456, 0.92559664]])

## Subscripted assignment for a ktensor

In [24]:
X.weights = np.ones(X.weights.size) # Insert new multipliers.
X

ktensor of shape 4 x 3 x 2
weights=[1. 1.]
factor_matrices[0] =
[[0.5488135  0.71518937]
 [0.60276338 0.54488318]
 [0.4236548  0.64589411]
 [0.43758721 0.891773  ]]
factor_matrices[1] =
[[0.96366276 0.38344152]
 [0.79172504 0.52889492]
 [0.56804456 0.92559664]]
factor_matrices[2] =
[[0.07103606 0.0871293 ]
 [0.0202184  0.83261985]]

## TODO: Why does subbing with `X[2][:, [0]] = ...` *not* work but `X.factor_matrices[2][:, [0]] = ...` does?

In [25]:
X.weights[0] = 7 # Change a single element of weights.

In [26]:
X.factor_matrices[2][:, [0]] = np.ones((2,1)) # Change the matrix for mode 3.
X

ktensor of shape 4 x 3 x 2
weights=[7. 1.]
factor_matrices[0] =
[[0.5488135  0.71518937]
 [0.60276338 0.54488318]
 [0.4236548  0.64589411]
 [0.43758721 0.891773  ]]
factor_matrices[1] =
[[0.96366276 0.38344152]
 [0.79172504 0.52889492]
 [0.56804456 0.92559664]]
factor_matrices[2] =
[[1.         0.0871293 ]
 [1.         0.83261985]]

## TODO: Not sure how to use end() here

## Use end for the last array index

In [27]:
X[1:2,X.end(1):,:]

array([2.44071817, 2.8167004 ])

In [28]:
X[0][0][1:X.end()] # Calculates X[0][0][1]

array([0.71518937])

### TODO: This doesn't work

In [29]:
# X[X.end()]

## Adding and subtracting ktensors

Adding two ktensors is the same as concatenating the matrices.

In [30]:
np.random.seed(0)
X = ttb.ktensor.from_factor_matrices([np.random.rand(4,2), np.random.rand(2,2),np.random.rand(3,2)]) # Data.
Y = ttb.ktensor.from_factor_matrices([np.random.rand(4,2), np.random.rand(2,2),np.random.rand(3,2)]) # More data.
X, Y

(ktensor of shape 4 x 2 x 3
 weights=[1. 1.]
 factor_matrices[0] =
 [[0.5488135  0.71518937]
  [0.60276338 0.54488318]
  [0.4236548  0.64589411]
  [0.43758721 0.891773  ]]
 factor_matrices[1] =
 [[0.96366276 0.38344152]
  [0.79172504 0.52889492]]
 factor_matrices[2] =
 [[0.56804456 0.92559664]
  [0.07103606 0.0871293 ]
  [0.0202184  0.83261985]],
 ktensor of shape 4 x 2 x 3
 weights=[1. 1.]
 factor_matrices[0] =
 [[0.77815675 0.87001215]
  [0.97861834 0.79915856]
  [0.46147936 0.78052918]
  [0.11827443 0.63992102]]
 factor_matrices[1] =
 [[0.14335329 0.94466892]
  [0.52184832 0.41466194]]
 factor_matrices[2] =
 [[0.26455561 0.77423369]
  [0.45615033 0.56843395]
  [0.0187898  0.6176355 ]])

In [31]:
Z = X + Y # Concatenates the factor matrices.
Z

ktensor of shape 4 x 2 x 3
weights=[1. 1. 1. 1.]
factor_matrices[0] =
[[0.5488135  0.71518937 0.77815675 0.87001215]
 [0.60276338 0.54488318 0.97861834 0.79915856]
 [0.4236548  0.64589411 0.46147936 0.78052918]
 [0.43758721 0.891773   0.11827443 0.63992102]]
factor_matrices[1] =
[[0.96366276 0.38344152 0.14335329 0.94466892]
 [0.79172504 0.52889492 0.52184832 0.41466194]]
factor_matrices[2] =
[[0.56804456 0.92559664 0.26455561 0.77423369]
 [0.07103606 0.0871293  0.45615033 0.56843395]
 [0.0202184  0.83261985 0.0187898  0.6176355 ]]

In [32]:
Z = X - Y # Concatenates as with plus, but changes the weights.
Z

ktensor of shape 4 x 2 x 3
weights=[ 1.  1. -1. -1.]
factor_matrices[0] =
[[0.5488135  0.71518937 0.77815675 0.87001215]
 [0.60276338 0.54488318 0.97861834 0.79915856]
 [0.4236548  0.64589411 0.46147936 0.78052918]
 [0.43758721 0.891773   0.11827443 0.63992102]]
factor_matrices[1] =
[[0.96366276 0.38344152 0.14335329 0.94466892]
 [0.79172504 0.52889492 0.52184832 0.41466194]]
factor_matrices[2] =
[[0.56804456 0.92559664 0.26455561 0.77423369]
 [0.07103606 0.0871293  0.45615033 0.56843395]
 [0.0202184  0.83261985 0.0187898  0.6176355 ]]

In [33]:
(Z.full() - (X.full() - Y.full())).norm() # Should be zero.

1.7333351416399796e-16

## Basic operations with a ktensor

In [34]:
+X # Calls uplus.

ktensor of shape 4 x 2 x 3
weights=[1. 1.]
factor_matrices[0] =
[[0.5488135  0.71518937]
 [0.60276338 0.54488318]
 [0.4236548  0.64589411]
 [0.43758721 0.891773  ]]
factor_matrices[1] =
[[0.96366276 0.38344152]
 [0.79172504 0.52889492]]
factor_matrices[2] =
[[0.56804456 0.92559664]
 [0.07103606 0.0871293 ]
 [0.0202184  0.83261985]]

In [35]:
-X # Calls uminus.

ktensor of shape 4 x 2 x 3
weights=[-1. -1.]
factor_matrices[0] =
[[0.5488135  0.71518937]
 [0.60276338 0.54488318]
 [0.4236548  0.64589411]
 [0.43758721 0.891773  ]]
factor_matrices[1] =
[[0.96366276 0.38344152]
 [0.79172504 0.52889492]]
factor_matrices[2] =
[[0.56804456 0.92559664]
 [0.07103606 0.0871293 ]
 [0.0202184  0.83261985]]

In [36]:
5*X # Calls mtimes.

ktensor of shape 4 x 2 x 3
weights=[5. 5.]
factor_matrices[0] =
[[0.5488135  0.71518937]
 [0.60276338 0.54488318]
 [0.4236548  0.64589411]
 [0.43758721 0.891773  ]]
factor_matrices[1] =
[[0.96366276 0.38344152]
 [0.79172504 0.52889492]]
factor_matrices[2] =
[[0.56804456 0.92559664]
 [0.07103606 0.0871293 ]
 [0.0202184  0.83261985]]

## Use permute to reorder the modes of a ktensor

In [37]:
X.permute(np.array((1,2,0))) # Reorders modes of X.

ktensor of shape 2 x 3 x 4
weights=[1. 1.]
factor_matrices[0] =
[[0.96366276 0.38344152]
 [0.79172504 0.52889492]]
factor_matrices[1] =
[[0.56804456 0.92559664]
 [0.07103606 0.0871293 ]
 [0.0202184  0.83261985]]
factor_matrices[2] =
[[0.5488135  0.71518937]
 [0.60276338 0.54488318]
 [0.4236548  0.64589411]
 [0.43758721 0.891773  ]]

## Use arrange to normalize the factors of a ktensor

The function `arrange` normalizes the columns of the factors and then arranges the rank-one pieces in decreasing order of size.

In [38]:
np.random.seed(0)
X = ttb.ktensor.from_factor_matrices([np.random.rand(3,2), np.random.rand(4,2),np.random.rand(2,2)]) # Unit weights.
X

ktensor of shape 3 x 4 x 2
weights=[1. 1.]
factor_matrices[0] =
[[0.5488135  0.71518937]
 [0.60276338 0.54488318]
 [0.4236548  0.64589411]]
factor_matrices[1] =
[[0.43758721 0.891773  ]
 [0.96366276 0.38344152]
 [0.79172504 0.52889492]
 [0.56804456 0.92559664]]
factor_matrices[2] =
[[0.07103606 0.0871293 ]
 [0.0202184  0.83261985]]

In [39]:
X.arrange() # Normalized and rearranged.
X

ktensor of shape 3 x 4 x 2
weights=[1.33623212 0.09761402]
factor_matrices[0] =
[[0.64602825 0.59738279]
 [0.49219122 0.65610716]
 [0.58343407 0.4611477 ]]
factor_matrices[1] =
[[0.61851988 0.3041712 ]
 [0.26594907 0.66985152]
 [0.36683329 0.55033591]
 [0.64197943 0.3948534 ]]
factor_matrices[2] =
[[0.10407646 0.96180106]
 [0.9945693  0.27374937]]

## Use fixsigns for sign indeterminacies in a ktensor

The largest magnitude entry for each factor is changed to be positive provided that we can flip the signs of pairs of vectors in that rank-1 component.

In [40]:
Y = X
Y.factor_matrices[0][:,0] = -Y.factor_matrices[0][:,0] # switch the sign on a pair of columns
Y.factor_matrices[1][:,0] = -Y.factor_matrices[1][:,0]
Y

ktensor of shape 3 x 4 x 2
weights=[1.33623212 0.09761402]
factor_matrices[0] =
[[-0.64602825  0.59738279]
 [-0.49219122  0.65610716]
 [-0.58343407  0.4611477 ]]
factor_matrices[1] =
[[-0.61851988  0.3041712 ]
 [-0.26594907  0.66985152]
 [-0.36683329  0.55033591]
 [-0.64197943  0.3948534 ]]
factor_matrices[2] =
[[0.10407646 0.96180106]
 [0.9945693  0.27374937]]

In [41]:
Y.fixsigns()

ktensor of shape 3 x 4 x 2
weights=[1.33623212 0.09761402]
factor_matrices[0] =
[[0.64602825 0.59738279]
 [0.49219122 0.65610716]
 [0.58343407 0.4611477 ]]
factor_matrices[1] =
[[0.61851988 0.3041712 ]
 [0.26594907 0.66985152]
 [0.36683329 0.55033591]
 [0.64197943 0.3948534 ]]
factor_matrices[2] =
[[0.10407646 0.96180106]
 [0.9945693  0.27374937]]

## Use ktensor to store the 'skinny' SVD of a matrix

In [42]:
np.random.seed(0)
A = np.random.rand(4,3)
A

array([[0.5488135 , 0.71518937, 0.60276338],
       [0.54488318, 0.4236548 , 0.64589411],
       [0.43758721, 0.891773  , 0.96366276],
       [0.38344152, 0.79172504, 0.52889492]])

## TODO: This creation of a ktensor from the SVD of `A` doesn't seem to be producing a size compatible with the MATLAB version & `X.double()` is not reproducing `A` either.

In [43]:
[U,S,V] = np.linalg.svd(A, full_matrices=False) # Compute the SVD.
X = ttb.ktensor.from_factor_matrices([np.diag(S), U, V]) # Store the SVD as a ktensor.
X

ktensor of shape 3 x 4 x 3
weights=[1. 1. 1.]
factor_matrices[0] =
[[2.21425506 0.         0.        ]
 [0.         0.29626672 0.        ]
 [0.         0.         0.21874633]]
factor_matrices[1] =
[[-0.48665747 -0.08953811  0.54332309]
 [-0.41275168 -0.81483266  0.00407301]
 [-0.62010377  0.2250203  -0.73183668]
 [-0.45636813  0.52668447  0.41133746]]
factor_matrices[2] =
[[-0.42376582 -0.64907895 -0.63175869]
 [-0.65045995  0.70346017 -0.2864361 ]
 [ 0.63033672  0.28955189 -0.72030224]]

In [44]:
X.double() # Reassemble the original matrix.

array([[[ 4.56643170e-01,  7.00925083e-01, -6.79240624e-01],
        [ 3.87295475e-01,  5.94479740e-01, -5.76088371e-01],
        [ 5.81859250e-01,  8.93125683e-01, -8.65495130e-01],
        [ 4.28221900e-01,  6.57299814e-01, -6.36964986e-01]],

       [[ 1.72182221e-02, -1.86608015e-02, -7.68098985e-03],
        [ 1.56692723e-01, -1.69820774e-01, -6.99000865e-02],
        [-4.32715152e-02,  4.68968951e-02,  1.93032745e-02],
        [-1.01281683e-01,  1.09767279e-01,  4.51814116e-02]],

       [[-7.50844786e-02, -3.40429120e-02, -8.56078733e-02],
        [-5.62868845e-04, -2.55201806e-04, -6.41757199e-04],
        [ 1.01136096e-01,  4.58545796e-02,  1.15310730e-01],
        [-5.68447383e-02, -2.57731086e-02, -6.48117593e-02]]])

## TODO: I couldn't seem to find a `datadisp` equivalent

## Displaying data

In [45]:
np.random.seed(0)
A = np.random.rand(4,2) # Create some data.
B = np.random.rand(3,2)
C = np.random.rand(2,2)
weights = np.array([5.0, 0.25])
X = ttb.ktensor.from_data(weights, [A,B,C]) # Create the ktensor.
X

ktensor of shape 4 x 3 x 2
weights=[5.   0.25]
factor_matrices[0] =
[[0.5488135  0.71518937]
 [0.60276338 0.54488318]
 [0.4236548  0.64589411]
 [0.43758721 0.891773  ]]
factor_matrices[1] =
[[0.96366276 0.38344152]
 [0.79172504 0.52889492]
 [0.56804456 0.92559664]]
factor_matrices[2] =
[[0.07103606 0.0871293 ]
 [0.0202184  0.83261985]]

In [46]:
print(X)

ktensor of shape 4 x 3 x 2
weights=[5.   0.25]
factor_matrices[0] =
[[0.5488135  0.71518937]
 [0.60276338 0.54488318]
 [0.4236548  0.64589411]
 [0.43758721 0.891773  ]]
factor_matrices[1] =
[[0.96366276 0.38344152]
 [0.79172504 0.52889492]
 [0.56804456 0.92559664]]
factor_matrices[2] =
[[0.07103606 0.0871293 ]
 [0.0202184  0.83261985]]
